# Binance Bot

I'm trying to build a machine learning bot for trading. I'm using the exchange binance

## Imports

Here I import each module or library needed for the anlysis and computations of the bot

In [1]:
import json
import talib
import os
import numpy as np
import pandas as pd
import random as rd
from binance.client import Client
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

import model_selection as ms

First of all, I want to try the algorithmic trading with one symbol, so here I connect myself with the API and perform teh extraction of one symbol for the last 35 days

In [2]:
with open('./params.json') as file: 
    params = json.load(file)

# Crea una instancia del cliente de Binance
client = Client(params["Api Key"], params["Secret Key"])

# Símbolo que deseas consultar
symbol = "BTCUSDT"

# Intervalo de tiempo (5 minutos)
interval = Client.KLINE_INTERVAL_5MINUTE

# Cuanto tiempo quiero coger
timeline = "42 day ago UTC"

Here I model the data for being more understandable for me

In [3]:
# Índice de la llamada a la API
columns = ["open_time", "open", "high", "low", "close", 
           "volume", "close_time", "quote_asset_volume", "number_of_trades", 
           "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"]

# Columnas que quiero dropear
dropColumns = ["open_time", "close_time", "quote_asset_volume", "taker_buy_base_asset_volume", 
               "taker_buy_quote_asset_volume", "ignore"]

# Obtiene los datos de cierre (Open time, Open, High, Low, Close, Volume y Close time)
candlesticks = pd.DataFrame(client.get_historical_klines(symbol, interval, timeline), columns=columns)
candlesticks.drop(columns=dropColumns, inplace=True)

#Convertir df a float
candlesticks = candlesticks.astype(float)

candlesticks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12096 entries, 0 to 12095
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   open              12096 non-null  float64
 1   high              12096 non-null  float64
 2   low               12096 non-null  float64
 3   close             12096 non-null  float64
 4   volume            12096 non-null  float64
 5   number_of_trades  12096 non-null  float64
dtypes: float64(6)
memory usage: 567.1 KB


## Computation of oscilator indicators

In [4]:
candlesticks["rsi"] = talib.RSI(candlesticks["close"], timeperiod= 14)
candlesticks["slowk"], candlesticks["slowd"] = talib.STOCH(candlesticks["high"], candlesticks["low"], candlesticks["close"])
candlesticks["willr"] = talib.WILLR(candlesticks["high"], candlesticks["low"], candlesticks["close"])
candlesticks["macd"], candlesticks["macdsignal"], candlesticks["macdhist"] = talib.MACD(candlesticks["close"])
candlesticks["aroon_up"], candlesticks["aroon_down"] = talib.AROON(candlesticks["high"], candlesticks["low"])
candlesticks["chaikin"] = talib.ADOSC(candlesticks["high"], candlesticks["low"], candlesticks["close"], candlesticks["volume"])
candlesticks["ultosc"] = talib.ULTOSC(candlesticks["high"], candlesticks["low"], candlesticks["close"])
candlesticks

,open,high,low,close,volume,number_of_trades,rsi,slowk,slowd,willr,macd,macdsignal,macdhist,aroon_up,aroon_down,chaikin,ultosc
0,16792.92,16796.62,16783.77,16784.01,316.24987,9865.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16783.90,16799.27,16781.80,16788.51,506.17361,13081.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16788.51,16792.20,16786.09,16787.29,295.34938,10085.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16787.56,16791.75,16781.00,16786.05,386.15064,9336.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16786.01,16792.47,16785.41,16789.60,404.33300,12751.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12091,23434.39,23438.04,23397.63,23409.59,739.58500,20622.0,55.119494,56.185988,61.610594,-20.987017,9.732397,2.219769,7.512628,35.714286,100.000000,395.868617,59.402987
12092,23410.14,23430.00,23407.58,23422.26,459.72212,13968.0,57.293155,75.388980,66.086643,-11.640602,11.286656,4.033147,7.253509,28.571429,92.857143,366.111358,60.596110
12093,23422.26,23439.57,23421.39,23428.26,440.44164,14672.0,58.322578,70.362342,67.312437,-8.250055,12.854389,5.797395,7.056994,21.428571,100.000000,286.429346,59.941083
12094,23427.73,23443.36,23402.92,23414.45,567.91889,17089.0,55.034381,71.337895,72.363072,-20.521011,12.834529,7.204822,5.629707,14.285714,100.000000,150.132606,55.974500


In [5]:
candlesticks["returns"] = candlesticks["close"].pct_change()
candlesticks

,open,high,low,close,volume,number_of_trades,rsi,slowk,slowd,willr,macd,macdsignal,macdhist,aroon_up,aroon_down,chaikin,ultosc,returns
0,16792.92,16796.62,16783.77,16784.01,316.24987,9865.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16783.90,16799.27,16781.80,16788.51,506.17361,13081.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000268
2,16788.51,16792.20,16786.09,16787.29,295.34938,10085.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000073
3,16787.56,16791.75,16781.00,16786.05,386.15064,9336.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000074
4,16786.01,16792.47,16785.41,16789.60,404.33300,12751.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12091,23434.39,23438.04,23397.63,23409.59,739.58500,20622.0,55.119494,56.185988,61.610594,-20.987017,9.732397,2.219769,7.512628,35.714286,100.000000,395.868617,59.402987,-0.001058
12092,23410.14,23430.00,23407.58,23422.26,459.72212,13968.0,57.293155,75.388980,66.086643,-11.640602,11.286656,4.033147,7.253509,28.571429,92.857143,366.111358,60.596110,0.000541
12093,23422.26,23439.57,23421.39,23428.26,440.44164,14672.0,58.322578,70.362342,67.312437,-8.250055,12.854389,5.797395,7.056994,21.428571,100.000000,286.429346,59.941083,0.000256
12094,23427.73,23443.36,23402.92,23414.45,567.91889,17089.0,55.034381,71.337895,72.363072,-20.521011,12.834529,7.204822,5.629707,14.285714,100.000000,150.132606,55.974500,-0.000589


In [6]:
candlesticks["target"] = np.where(candlesticks["returns"] > 0, 1, 0)
candlesticks["target"] = candlesticks["target"].shift(-1)
candlesticks.head(10)

,open,high,low,close,volume,number_of_trades,rsi,slowk,slowd,willr,macd,macdsignal,macdhist,aroon_up,aroon_down,chaikin,ultosc,returns,target
0,16792.92,16796.62,16783.77,16784.01,316.24987,9865.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,16783.90,16799.27,16781.80,16788.51,506.17361,13081.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000268,0.0
2,16788.51,16792.20,16786.09,16787.29,295.34938,10085.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000073,0.0
3,16787.56,16791.75,16781.00,16786.05,386.15064,9336.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000074,1.0
4,16786.01,16792.47,16785.41,16789.60,404.33300,12751.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000211,1.0
5,16789.60,16797.69,16788.62,16792.65,443.49631,11190.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000182,0.0
6,16792.77,16798.70,16775.58,16780.88,464.02946,11425.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000701,1.0
7,16780.37,16791.10,16779.38,16783.87,322.29019,9265.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000178,0.0
8,16783.87,16784.72,16774.27,16780.73,472.26226,10878.0,NaN,28.407725,37.947834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000187,0.0
9,16780.92,16783.35,16772.44,16776.81,269.07250,8579.0,NaN,26.313526,31.856641,NaN,NaN,NaN,NaN,NaN,NaN,-153.94081,NaN,-0.000234,1.0


## Cleaning data

Before passing the data to the model we have to do some cleaning

In [7]:
data = candlesticks.copy()
columnsToDrop = ["open", "close", "high", "low", "number_of_trades", "volume"]
data.drop(columns = columnsToDrop, inplace = True)
data.dropna(inplace = True)
data

,rsi,slowk,slowd,willr,macd,macdsignal,macdhist,aroon_up,aroon_down,chaikin,ultosc,returns,target
33,62.517707,76.114138,64.673275,-6.412406,8.231623,8.956884,-0.725261,64.285714,100.000000,144.980019,59.272646,0.000176,1.0
34,64.690637,92.169272,76.354109,-6.907433,8.776317,8.920770,-0.144454,57.142857,100.000000,249.565076,63.010461,0.000251,0.0
35,55.971266,76.119664,81.467691,-44.701415,8.263056,8.789227,-0.526171,50.000000,100.000000,174.919772,57.593790,-0.000626,0.0
36,51.226008,51.796104,73.361680,-67.897825,7.230695,8.477521,-1.246826,42.857143,92.857143,44.083788,50.573818,-0.000400,0.0
37,50.593605,28.012962,51.976243,-71.073524,6.266073,8.035231,-1.769158,35.714286,85.714286,28.959079,55.382801,-0.000055,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12090,60.738345,66.684960,71.480260,-1.205812,8.897072,0.341612,8.555460,42.857143,100.000000,491.319218,64.211008,0.001989,0.0
12091,55.119494,56.185988,61.610594,-20.987017,9.732397,2.219769,7.512628,35.714286,100.000000,395.868617,59.402987,-0.001058,1.0
12092,57.293155,75.388980,66.086643,-11.640602,11.286656,4.033147,7.253509,28.571429,92.857143,366.111358,60.596110,0.000541,1.0
12093,58.322578,70.362342,67.312437,-8.250055,12.854389,5.797395,7.056994,21.428571,100.000000,286.429346,59.941083,0.000256,0.0


## Training and testing division

In [8]:
division = int(0.83 * len(data))
train = data.iloc[:division]
test = data.iloc[division:]

print(f"Training shape: {train.shape}")
print(f"Testing shape: {test.shape}")

Training shape: (10011, 13)
Testing shape: (2051, 13)


## Feature selection

half = int(len(test))
featureSelectionData = test.iloc[:half].copy()
testingData = test.iloc[half:]

random = rd.randint(1, 100)

model = XGBClassifier(gamma=1.5, n_estimators=25,
                      learning_rate=0.03, max_depth=3,
                      objective = 'binary:logistic', eval_metric = 'auc', 
                      booster = 'gbtree', subsample=0.8, colsample_bytree=1,
                      use_label_encoder = False, random_state=random) 

divsion = int(len(featureSelectionData) * 0.9)

trainFS = featureSelectionData.iloc[:divsion]
testFS = featureSelectionData.iloc[divsion:]

X_trainFS = trainFS.drop('target',axis=1)
X_testFS = testFS.drop('target',axis=1)
y_trainFS = trainFS['target']
y_testFS = testFS['target']

model = model.fit(X_trainFS, y_trainFS)
prediction = model.predict(X_testFS)

accuracy = accuracy_score(y_testFS, prediction)

print(f"Accuracy: {accuracy}")

#Plot Feature Importance
fig = plt.figure(figsize=(22,5))
importance_labels = X_trainFS.columns
importance_features = model.feature_importances_
plt.bar(importance_labels, importance_features)
plt.show()

mean_feature_importance = importance_features.mean()
treshold = mean_feature_importance - mean_feature_importance*0.2

recommended_feature_labels = []
recommended_feature_score = []
for i in range (len(importance_features)): 
    if importance_features[i] > treshold:
        recommended_feature_labels.append(importance_labels[i])
        recommended_feature_score.append(importance_features[i])

#Plot recommended features
fig = plt.figure(figsize=(15,5))
plt.bar(recommended_feature_labels, recommended_feature_score)
plt.show()

recommended_feature_labels.append('target')
featuredSelectedData = featureSelectionData[recommended_feature_labels].copy()

model = XGBClassifier(gamma=1.5, n_estimators=25,
                      learning_rate=0.03, max_depth=3,
                      objective = 'binary:logistic', eval_metric = 'auc', 
                      booster = 'gbtree', subsample=0.8, colsample_bytree= 1,
                      use_label_encoder = False, random_state=random)

divsion = int(len(featureSelectionData) * 0.9)

trainAFS = featuredSelectedData.iloc[:divsion]
testAFS = featuredSelectedData.iloc[divsion:]

X_trainAFS = trainAFS.drop('target',axis=1)
X_testAFS = testAFS.drop('target',axis=1)
y_trainAFS = trainAFS['target']
y_testAFS = testAFS['target']

model = model.fit(X_trainAFS, y_trainAFS)
prediction = model.predict(X_testAFS)

accuracy = accuracy_score(y_testAFS, prediction)

print(f"Accuracy: {accuracy}")

## 10f - CV

In [10]:
# selectedData = train[recommended_feature_labels].copy()
selectedData = train.copy()

folds = []
nfolds = 10
foldSize = int(1/nfolds * len(selectedData))

for i in range(nfolds):
    if i != nfolds-1: folds.append(selectedData[i*foldSize:(i+1)*foldSize])
    else: folds.append(selectedData[i*foldSize:])
        
for i in range (len(folds)):
    folds[i].to_csv(f"./folds/fold_{i}.csv", index=False)
    
print(f"Folds shape: {folds[0].shape}")

Folds shape: (1001, 13)


## Model selection

In [11]:
#Exporting folds
folder = 'folds'
folds = []
for file in os.listdir(folder):
    fold = pd.read_csv(f'{folder}/{file}')
    folds.append(fold)

print(f"FOLD SHAPE: {folds[0].shape}")
print()
print(fold)

# Find best params and saving data
best_model, best_accuracy = ms.find_best_params(folds)

FOLD SHAPE: (1001, 13)

            rsi      slowk      slowd      willr      macd  macdsignal  \
0     62.517707  76.114138  64.673275  -6.412406  8.231623    8.956884   
1     64.690637  92.169272  76.354109  -6.907433  8.776317    8.920770   
2     55.971266  76.119664  81.467691 -44.701415  8.263056    8.789227   
3     51.226008  51.796104  73.361680 -67.897825  7.230695    8.477521   
4     50.593605  28.012962  51.976243 -71.073524  6.266073    8.035231   
...         ...        ...        ...        ...       ...         ...   
996   45.660478  45.544249  39.918411 -36.904762 -6.391349   -7.087890   
997   37.638293  38.357630  40.966822 -93.430335 -6.811698   -7.032652   
998   34.173371  35.231572  39.711151 -66.714218 -7.611439   -7.148409   
999   40.879226  31.133822  34.907675 -46.682927 -7.524268   -7.223581   
1000  49.643856  57.560615  41.308670 -16.268293 -6.375467   -7.053958   

      macdhist    aroon_up  aroon_down     chaikin     ultosc   returns  \
0    -0.7252

Parameters: Gamma=0.5, max_depth:3, n_estimators:25, learning_rate:0.02
MODEL: XGBClassifier(eval_metric='auc', gamma=0.5, learning_rate=0.02, n_estimators=25,
              random_state=44, subsample=0.8, use_label_encoder=False)
Mean accuracy: 0.5326732673267326
-------------------------
Parameters: Gamma=0.5, max_depth:3, n_estimators:25, learning_rate:0.05
MODEL: XGBClassifier(eval_metric='auc', gamma=0.5, learning_rate=0.05, n_estimators=25,
              random_state=29, subsample=0.8, use_label_encoder=False)
Mean accuracy: 0.5326732673267326
-------------------------
Parameters: Gamma=0.5, max_depth:3, n_estimators:25, learning_rate:0.1
MODEL: XGBClassifier(eval_metric='auc', gamma=0.5, n_estimators=25, random_state=57,
              subsample=0.8, use_label_encoder=False)
Mean accuracy: 0.5297029702970297
-------------------------
Parameters: Gamma=0.5, max_depth:3, n_estimators:25, learning_rate:0.2
MODEL: XGBClassifier(eval_metric='auc', gamma=0.5, learning_rate=0.2, n_estim

Parameters: Gamma=0.5, max_depth:4, n_estimators:50, learning_rate:0.2
MODEL: XGBClassifier(eval_metric='auc', gamma=0.5, learning_rate=0.2, max_depth=4,
              n_estimators=50, random_state=71, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5059405940594061
-------------------------
Parameters: Gamma=0.5, max_depth:4, n_estimators:100, learning_rate:0.01
MODEL: XGBClassifier(eval_metric='auc', gamma=0.5, learning_rate=0.01, max_depth=4,
              random_state=59, subsample=0.8, use_label_encoder=False)
Mean accuracy: 0.5396039603960395
-------------------------
Parameters: Gamma=0.5, max_depth:4, n_estimators:100, learning_rate:0.02
MODEL: XGBClassifier(eval_metric='auc', gamma=0.5, learning_rate=0.02, max_depth=4,
              random_state=73, subsample=0.8, use_label_encoder=False)
Mean accuracy: 0.5267326732673269
-------------------------
Parameters: Gamma=0.5, max_depth:4, n_estimators:100, learning_rate:0.05
MODEL: XGBClassifier(eval_metric='a

Parameters: Gamma=0.5, max_depth:5, n_estimators:150, learning_rate:0.05
MODEL: XGBClassifier(eval_metric='auc', gamma=0.5, learning_rate=0.05, max_depth=5,
              n_estimators=150, random_state=69, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.50990099009901
-------------------------
Parameters: Gamma=0.5, max_depth:5, n_estimators:150, learning_rate:0.1
MODEL: XGBClassifier(eval_metric='auc', gamma=0.5, max_depth=5, n_estimators=150,
              random_state=10, subsample=0.8, use_label_encoder=False)
Mean accuracy: 0.5297029702970297
-------------------------
Parameters: Gamma=0.5, max_depth:5, n_estimators:150, learning_rate:0.2
MODEL: XGBClassifier(eval_metric='auc', gamma=0.5, learning_rate=0.2, max_depth=5,
              n_estimators=150, random_state=86, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5118811881188119
-------------------------
Parameters: Gamma=0.5, max_depth:6, n_estimators:25, learning_rate:0.01
MODEL: 

Parameters: Gamma=1, max_depth:2, n_estimators:50, learning_rate:0.01
MODEL: XGBClassifier(eval_metric='auc', gamma=1, learning_rate=0.01, max_depth=2,
              n_estimators=50, random_state=78, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5198019801980198
-------------------------
Parameters: Gamma=1, max_depth:2, n_estimators:50, learning_rate:0.02
MODEL: XGBClassifier(eval_metric='auc', gamma=1, learning_rate=0.02, max_depth=2,
              n_estimators=50, random_state=36, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5188118811881187
-------------------------
Parameters: Gamma=1, max_depth:2, n_estimators:50, learning_rate:0.05
MODEL: XGBClassifier(eval_metric='auc', gamma=1, learning_rate=0.05, max_depth=2,
              n_estimators=50, random_state=47, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5267326732673266
-------------------------
Parameters: Gamma=1, max_depth:2, n_estimators:50, learnin

...

Parameters: Gamma=2, max_depth:5, n_estimators:25, learning_rate:0.01
MODEL: XGBClassifier(eval_metric='auc', gamma=2, learning_rate=0.01, max_depth=5,
              n_estimators=25, random_state=63, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5198019801980198
-------------------------
Parameters: Gamma=2, max_depth:5, n_estimators:25, learning_rate:0.02
MODEL: XGBClassifier(eval_metric='auc', gamma=2, learning_rate=0.02, max_depth=5,
              n_estimators=25, random_state=69, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5396039603960396
-------------------------
Parameters: Gamma=2, max_depth:5, n_estimators:25, learning_rate:0.05
MODEL: XGBClassifier(eval_metric='auc', gamma=2, learning_rate=0.05, max_depth=5,
              n_estimators=25, random_state=20, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5386138613861385
-------------------------
Parameters: Gamma=2, max_depth:5, n_estimators:25, learnin

Parameters: Gamma=2, max_depth:6, n_estimators:50, learning_rate:0.1
MODEL: XGBClassifier(eval_metric='auc', gamma=2, max_depth=6, n_estimators=50,
              random_state=97, subsample=0.8, use_label_encoder=False)
Mean accuracy: 0.5247524752475248
-------------------------
Parameters: Gamma=2, max_depth:6, n_estimators:50, learning_rate:0.2
MODEL: XGBClassifier(eval_metric='auc', gamma=2, learning_rate=0.2, max_depth=6,
              n_estimators=50, random_state=83, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5207920792079209
-------------------------
Parameters: Gamma=2, max_depth:6, n_estimators:100, learning_rate:0.01
MODEL: XGBClassifier(eval_metric='auc', gamma=2, learning_rate=0.01, max_depth=6,
              random_state=45, subsample=0.8, use_label_encoder=False)
Mean accuracy: 0.5415841584158416
-------------------------
Parameters: Gamma=2, max_depth:6, n_estimators:100, learning_rate:0.02
MODEL: XGBClassifier(eval_metric='auc', gamma=2, learn

Parameters: Gamma=5, max_depth:2, n_estimators:150, learning_rate:0.02
MODEL: XGBClassifier(eval_metric='auc', gamma=5, learning_rate=0.02, max_depth=2,
              n_estimators=150, random_state=63, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5257425742574258
-------------------------
Parameters: Gamma=5, max_depth:2, n_estimators:150, learning_rate:0.05
MODEL: XGBClassifier(eval_metric='auc', gamma=5, learning_rate=0.05, max_depth=2,
              n_estimators=150, random_state=32, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.510891089108911
-------------------------
Parameters: Gamma=5, max_depth:2, n_estimators:150, learning_rate:0.1
MODEL: XGBClassifier(eval_metric='auc', gamma=5, max_depth=2, n_estimators=150,
              random_state=76, subsample=0.8, use_label_encoder=False)
Mean accuracy: 0.5198019801980198
-------------------------
Parameters: Gamma=5, max_depth:2, n_estimators:150, learning_rate:0.2
MODEL: XGBClassifi

Parameters: Gamma=5, max_depth:4, n_estimators:50, learning_rate:0.01
MODEL: XGBClassifier(eval_metric='auc', gamma=5, learning_rate=0.01, max_depth=4,
              n_estimators=50, random_state=54, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5178217821782178
-------------------------
Parameters: Gamma=5, max_depth:4, n_estimators:50, learning_rate:0.02
MODEL: XGBClassifier(eval_metric='auc', gamma=5, learning_rate=0.02, max_depth=4,
              n_estimators=50, random_state=69, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5316831683168317
-------------------------
Parameters: Gamma=5, max_depth:4, n_estimators:50, learning_rate:0.05
MODEL: XGBClassifier(eval_metric='auc', gamma=5, learning_rate=0.05, max_depth=4,
              n_estimators=50, random_state=22, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5267326732673266
-------------------------
Parameters: Gamma=5, max_depth:4, n_estimators:50, learnin

Parameters: Gamma=5, max_depth:5, n_estimators:100, learning_rate:0.1
MODEL: XGBClassifier(eval_metric='auc', gamma=5, max_depth=5, random_state=12,
              subsample=0.8, use_label_encoder=False)
Mean accuracy: 0.50990099009901
-------------------------
Parameters: Gamma=5, max_depth:5, n_estimators:100, learning_rate:0.2
MODEL: XGBClassifier(eval_metric='auc', gamma=5, learning_rate=0.2, max_depth=5,
              random_state=67, subsample=0.8, use_label_encoder=False)
Mean accuracy: 0.512871287128713
-------------------------
Parameters: Gamma=5, max_depth:5, n_estimators:150, learning_rate:0.01
MODEL: XGBClassifier(eval_metric='auc', gamma=5, learning_rate=0.01, max_depth=5,
              n_estimators=150, random_state=41, subsample=0.8,
              use_label_encoder=False)
Mean accuracy: 0.5316831683168317
-------------------------
Parameters: Gamma=5, max_depth:5, n_estimators:150, learning_rate:0.02
MODEL: XGBClassifier(eval_metric='auc', gamma=5, learning_rate=0.02, ma